In [ ]:
import pandas as pd
import numpy as np
import re
import os
import glob
from datetime import timedelta
from tqdm import tqdm

import requests
import datetime
import tempfile
import re
import pytz
import isodate
import seaborn as sns
from collections import Counter
from google.oauth2 import service_account
from googleapiclient.discovery import build
from google.cloud import bigquery,storage


## 유튜브 API


In [ ]:
# 채널id 가져오기
def get_channel_id(api_key, channel_name):
    url = f"https://www.googleapis.com/youtube/v3/search?part=snippet&q={channel_name}&type=channel&key={api_key}"
    response = requests.get(url)
    data = response.json()
    channel_Id = data["items"][0]["id"]["channelId"]
    return channel_Id

In [ ]:
# 모든 비디오 가져오기
def scroll(channel_Id):
      video_list = []
      # 스크롤이 되지 않을 때까지 nextpageToken 무한 호출
      try :
          res = youtube.channels().list(id=channel_Id, part='contentDetails').execute()
          # 플레이리스트 가져오기
          playlist_id = res['items'][0]['contentDetails']['relatedPlaylists']['uploads']

          next_page = None
          # 영상 n개 까지 수집
          while len(video_list) < 1200:
              # 다음 페이지의 Token 반환
              res = youtube.playlistItems().list(playlistId=playlist_id,part='snippet',maxResults=50,pageToken=next_page).execute()
              video_list += res['items']
              next_page = res.get('nextPageToken')

              if next_page is None :
                  break
          return video_list

      except Exception as e:
          print('API 호출 한도 초과') # API 할당량 초과 예외처리

          return video_list


# 영상별 조회수 및 좋아요 static
def get_static(video_ids, api_key):
    url_template = 'https://www.googleapis.com/youtube/v3/videos?id={}&key={}&part=statistics'

    result_static = []
    for video_id in video_ids:
        url = url_template.format(video_id, api_key)
        response = requests.get(url)
        static = response.json()
        result_static.append(static)

    video_ids = []
    view_counts = []
    like_counts = []
    comment_counts = []

    for item in result_static:
        try:
            video_id = item['items'][0]['id']
            view_count = item['items'][0]['statistics']['viewCount']
            like_count = item['items'][0]['statistics']['likeCount']
            comment_count = item['items'][0]['statistics']['commentCount']
        except (KeyError,IndexError):
            continue
        video_ids.append(video_id)
        view_counts.append(view_count)
        like_counts.append(like_count)
        comment_counts.append(comment_count)

    static_df = pd.DataFrame({'video_id': video_ids,
                'view_count': view_counts,
                'like_count': like_counts,
                'comment_count': comment_counts,
                                })

    static_df['view_count'] = static_df['view_count'].astype(int)
    static_df['like_count'] = static_df['like_count'].astype(int)
    static_df['comment_count'] = static_df['comment_count'].astype(int)
    return static_df

def to_dataframe(video_list):
  data = []
  for item in video_list:
      snippet     = item['snippet']
      publishedAt = snippet['publishedAt']
      title       = snippet['title']
      video_id    = snippet['resourceId']['videoId']

      try:  # thumbnail 값이 없을 때 passs
          thumbnail = snippet['thumbnails']['maxres']['url']
      except KeyError:
          thumbnail = f'https://i.ytimg.com/vi/{video_id}/0.jpg'

      data.append({'publishedAt' : publishedAt,'title': title,'video_id': video_id,'thumbnails':thumbnail})
  video_df = pd.DataFrame(data)
  video_ids = video_df['video_id'].tolist()
  return video_df, video_ids



# 영상의 길이
def video_duration(video_ids, api_key):
    data = []
    for video_id in video_ids:
        url = f"https://www.googleapis.com/youtube/v3/videos?part=contentDetails&id={video_id}&key={api_key}"
        response = requests.get(url)
        video_data = response.json()

        try:
            duration = video_data['items'][0]['contentDetails']['duration']
            data.append({'video_id': video_id, 'Duration': duration})
        except KeyError:
            data.append({'video_id': video_id, 'Duration': 'Unknown'})

    # DataFrame 생성
    duration_df = pd.DataFrame(data)
    return duration_df

def iso_to_seconds(iso_duration):
    time_delta = isodate.parse_duration(iso_duration)
    return int(time_delta.total_seconds())



In [ ]:
# 댓글 데이터 수집
def comment(videoIds, youtube):
    all_comments = []

    for videoId in videoIds:
        comments = []
        try:
            response = youtube.commentThreads().list(part='snippet,replies', videoId=videoId, maxResults=100).execute()

            while response:
                for item in response['items']:
                    # 댓글 정보
                    comment = item['snippet']['topLevelComment']['snippet']
                    comments.append([comment['authorDisplayName'], comment['authorChannelId']['value'], comment['textDisplay'], comment['likeCount'], comment['publishedAt']])

                    # # 답글 정보
                    # if item['snippet']['totalReplyCount'] > 0:
                    #     for reply_item in item['replies']['comments']:
                    #         reply = reply_item['snippet']
                    #         comments.append([reply['authorDisplayName'], reply['authorChannelId']['value'], reply['textDisplay'], reply['likeCount'], reply['publishedAt']])

                # 다음 페이지가 있으면 다음 페이지로 이동
                if 'nextPageToken' in response:
                    response = youtube.commentThreads().list(part='snippet,replies', videoId=videoId, pageToken=response['nextPageToken'], maxResults=100).execute()
                else:
                    break

            # DataFrame 생성
            comment_df = pd.DataFrame(comments, columns=['author', 'author_channel_id', 'comment', 'like', 'publishedAt'])
            comment_df['video_id'] = videoId
            all_comments.append(comment_df)

        except HttpError as e:
            error_reason = e.error_details[0].get('reason', '')
            if e.resp.status == 403 and error_reason == 'quotaExceeded':
                print("Quota exceeded. Returning collected comments.")
                # 할당량 초과 시 수집된 데이터 반환
                if all_comments:
                    return pd.concat(all_comments, ignore_index=True).rename(columns={'publishedAt': 'commentAt'})
                else:
                    return pd.DataFrame()  # 빈 DataFrame 반환
            else:
                print(f"Error with videoId {videoId}: {e}")
                continue  # 다음 videoId로 이동


    # 모든 비디오의 댓글을 하나의 DataFrame으로 병합
    if all_comments:
        comment_df = pd.concat(all_comments, ignore_index=True)
        comment_df = comment_df.rename(columns={'publishedAt': 'commentAt'})
        return comment_df
    else:
        print("No comments found.")
        return pd.DataFrame()  # 빈 DataFrame 반환

### 본채널 영상수집


In [ ]:
# ---------------------------------------------------- 데이터 수집 --------------------------------------------------------------------------- #
channel_name = 'waktaverse'                      # 채널명 # woowakgood , waktaverse, woowakgoodzz, wakbanchan
channel_Id = get_channel_id(api_key, channel_name) # 채널 id
video_list = scroll(channel_Id)                    # 채널에 해당하는 모든 영상 수집
video_df, video_ids = to_dataframe(video_list)     # 수집된 json 들을 dataframe으로
static_df = get_static(video_ids, api_key)         # 수집된 video_id를 통해 static 값 수집
duration_df = video_duration(video_ids,api_key)    # 수집된 video_id를 통해 영상의 길이 수집
merged_df = pd.merge(static_df, video_df, on='video_id')    # 데이터 결합
total_df = pd.merge(duration_df, merged_df, on ='video_id') # 최종 데이터
total_df['channel'] = channel_name
total_df['seconds'] = total_df['Duration'].apply(iso_to_seconds)

# total_df['url'] = 'https://www.youtube.com/watch?v='+ total_df['video_id']
# total_df = total_df[['video_id','title','publishedAt','Duration','view_count','like_count','comment_count','thumbnails']]
# now = datetime.datetime.now()


In [ ]:
print(total_df['publishedAt'].min())
print(total_df['publishedAt'].max())

2021-04-08T12:12:20Z
2024-12-14T17:50:57Z


In [ ]:
static_df = get_static(video_ids, api_key)
static_df

In [ ]:
# UTC를 KST로 변환 (UTC+9)
total_df['publishedAt'] = pd.to_datetime(total_df['publishedAt'], format='%Y-%m-%dT%H:%M:%SZ', utc=True)
total_df['publishedAt_KST'] = total_df['publishedAt'].dt.tz_convert('Asia/Seoul')

In [ ]:
# 집계에 필요한 데이터만
total_df['publishedAt_KST'] = pd.to_datetime(total_df['publishedAt_KST'])
start_date = '2023-12-01'
end_date = '2024-12-05'
filtered_df_wakta = total_df[(total_df['publishedAt_KST'] >= start_date) & (total_df['publishedAt_KST'] <= end_date)]

In [ ]:
filtered_df_wakta.to_csv('/content/drive/MyDrive/WAKTUBE/본채널_왁타버스_df.csv',index=False, encoding="utf-8-sig")

## 고정멤버 댓글 언급량 전처리

In [ ]:
!pip install --target=$my_path konlpy
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab

! git clone https://github.com/lovit/customized_konlpy.git
! pip install customized_konlpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.8/493.8 kB 38.6 MB/s eta 0:00:00
Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 138, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 138 (delta 26), reused 22 (delta 8), pack-reused 91 (from 1)
Receiving objects: 100% (138/138), 1.72 MiB | 4.38 MiB/s, done.
Resolving deltas: 100% (65/65), done.
/content/Mecab-ko-for-Google-Colab
Cloning into 'customized_konlpy'...
remote: Enumerating objects: 618, done.
remote: Total 618 (delta 0), reused 0 (delta 0), pack-reused 618 (from 1)
Receiving objects: 100% (618/618), 950.06 KiB | 2.79 MiB/s, done.
Resolving deltas: 100% (345/345), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 881.5/881.5 kB 15.5 MB/s eta 0:00:00


In [ ]:
wakgood = pd.read_csv('/content/drive/MyDrive/WAKTUBE/24_wakgood_comment_final.csv')
wakta = pd.read_csv('/content/drive/MyDrive/WAKTUBE/24_wakta_comment_final.csv')
Z = pd.read_csv('/content/drive/MyDrive/WAKTUBE/24_Z_comment_final.csv')
banchan = pd.read_csv('/content/drive/MyDrive/WAKTUBE/24_Banchan_final.csv')
comment_df = pd.concat([wakgood,wakta,Z,banchan])

In [ ]:
comment_df['commentAt'] = pd.to_datetime(comment_df['commentAt'])
comment_df['year'] = comment_df['commentAt'].dt.year
comment_df['month'] = comment_df['commentAt'].dt.month

고정멤버 단어사전

In [ ]:
word_rules  = {

    "뢴트게늄" : ['뢴트게늄','뢴트','초코푸딩'],
    "해루석" : ['루숙','해루석','해루숙','루석'],
    "캘리칼리": ['캘칼', '캘리칼리', '캘리칼리데이빈슨'],
    "도파민" :['도파민','파민','박사','할배즈'],
    "소피아" : ['소피아','춘피아'],
    "권민" : ['권민','쿤미옌'],
    "왁파고": ['왁파고', '파고', '황파고'],
    "독고혜지": ['혜지','혜디','독고혜지'],
    "비밀소녀": ['비소','비밀소녀','비밀이모'],
    "히키킹" : ['히키킹','히키퀸','히덩이'],
    "곽춘식" : ['춘식','곽춘식','춘피아','춘타버스'],
    "김치만두" : ['김치만두','만두','김치만두번영택사스가','대황두'],
    "티파니" : ['파니','티파니'],
    "비즈니스킴":['비킴','비즈니스킴'],
    "풍신" :['풍신','할배즈'],
    "바이터":['바이터','프리터'],
    "단답벌레" : ['단답벌레','단답','기획벌레'],
    "융터르" : ['카르나르','융털','융터르'],
    "호드" : ['호드','노스페라투스'],
    "이덕수" : ['덕수','이덕수','할배즈'],
    "부정형 인간":['부정형 인간','부정형'],


    "미미짱짱세용" : ['미미짱짱세용','세용','용언'],
    "닌닌" : ['닌닌'],
    '데스해머쵸로키':['초로키','쵸로키','로키','데스해머','초로키'],
    "젠투" : ['젠투','젠크리트','젠황'],
    "수셈이" : ['세미','셈이','수셈이','수세미'],
    "아마최" : ['아마데우스최','아마최','킹마최'],
    "진희" : ['진희','지니','치니'],
    "시리안": ['시리안','빡리안'],
    "길버트":['길버트','버트','길벝'],
    "빅토리":['빅토리','맑눈광','토리','빅듁'],
    "설리반":['설리반','리반이'],
    "불곰":['불곰'],
    "성기사 샬롯":['샬롯','성기사 샬롯'],
    "버터우스 3세" :['버터우스','버터'],
    "크리즈" :['크리즈','돌호동'],
    "아이스께끼": ['께황','깨황','아이스께끼','께끼','아이쓰께끼','아이스께끼','깨끼'],
    "메카 맹기산":['맹기산','기산이형','기사니','기싸니','기산']


}


gomem_word = {
    "뢴트게늄" : ['뢴트게늄','뢴트','초코푸딩'],
    "해루석" : ['루숙','해루석','해루숙','루석'],
    "캘리칼리": ['캘칼', '캘리칼리', '캘리칼리데이빈슨'],
    "도파민" :['도파민','파민','박사','할배즈'],
    "소피아" : ['소피아','춘피아'],
    "권민" : ['권민','쿤미옌'],
    "왁파고": ['왁파고', '파고', '황파고'],
    "독고혜지": ['혜지','혜디','독고혜지'],
    "비밀소녀": ['비소','비밀소녀','비밀이모'],
    "히키킹" : ['히키킹','히키퀸','히덩이'],
    "곽춘식" : ['춘식','곽춘식','춘피아','춘타버스'],
    "김치만두" : ['김치만두','만두','김치만두번영택사스가'],
    "티파니" : ['파니','티파니'],
    "비즈니스킴":['비킴','비즈니스킴'],
    "풍신" :['풍신','할배즈'],
    "바이터":['바이터','프리터'],
    "단답벌레" : ['단답벌레','단답','기획벌레'],
    "융터르" : ['카르나르','융털','융터르'],
    "호드" : ['호드','노스페라투스'],
    "이덕수" : ['덕수','이덕수','할배즈'],
    "부정형 인간":['부정형 인간','부정형'],
}

aka_word = {

    "미미짱짱세용" : ['미미짱짱세용','세용','용언'],
    "닌닌" : ['닌닌'],
    '데스해머쵸로키':['초로키','쵸로키','로키','데스해머','초로키'],
    "젠투" : ['젠투','젠크리트','젠황'],
    "수셈이" : ['세미','셈이','수셈이','수세미'],
    "아마최" : ['아마데우스최','아마최','킹마최'],
    "진희" : ['진희','지니','치니'],
    "시리안": ['시리안','빡리안'],
    "길버트":['길버트','버트','길벝'],
    "빅토리":['빅토리','맑눈광','토리','빅듁'],
    "설리반":['설리반','리반이'],
    "불곰":['불곰'],
    "성기사 샬롯":['샬롯','성기사 샬롯'],
    "버터우스 3세" :['버터우스','버터'],
    "크리즈" :['크리즈','돌호동'],
    "아이스께끼": ['께황','깨황','아이스께끼','께끼','아이쓰께끼','아이스께끼','깨끼'],
    "메카 맹기산":['맹기산','기산이형','기사니','기싸니','기산'],
    "썸네일":['썸네일',"썸내일",'썸넬','썸낼']

}

In [ ]:
from ckonlpy.tag import Twitter
from collections import Counter
from tqdm import tqdm

In [ ]:
comment_df['comment'] = comment_df['comment'].apply(lambda x: re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', '', x))
comment_df['comment'].replace('', np.nan, inplace=True)  #비어 있는 행은 null값으로 처리
comment_df.dropna(how='any', inplace=True)  #null 값 제거
comment_df.drop_duplicates(subset = ['comment'], inplace=True) #중복된 행 제거

<ipython-input-18-c71e8a5f7bd0>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  comment_df['comment'].replace('', np.nan, inplace=True)  #비어 있는 행은 null값으로 처리


In [ ]:
comment_df.sample(100)

In [ ]:
twi = Twitter()

gomem = ['뢴트게늄', '뢴트', '초코푸딩',
         '해루석', '해루숙', '루석','루숙',
         '캘칼', '캘리칼리', '캘리칼리데이빈슨',
         '도파민', '파민','파미니','박사', '할배즈',
         '소피아', '춘피아',
         '권민', '쿤미옌',
         '왁파고', '파고', '황파고',
         '혜지', '혜디', '독고혜지',
         '비소', '비밀소녀','비밀이모',
         '히키킹', '히키퀸','히덩이',
         '춘식', '곽춘식', '춘피아','춘타버스',
         '김치만두', '만두', '김치만두번영택사스가','대황두',
         '티파니', '파니','하쿠',
         '비킴', '비즈니스킴',
         '풍신', '할배즈',
         '바이터','프리터',
         '단답벌레', '단답','기획벌레',
         '카르나르', '융털', '융터르',
         '호드', '노스페라투스',
         '덕수', '이덕수', '할배즈',
         '부정형 인간','부정형'
        ]

akadam = ['미미짱짱세용', '세용','용언',
          '닌닌',
          '데스해머쵸로키','초로키','쵸로키','로키','데스해머','초로키',
          '젠투', '젠크리트', '젠황',
          '세미', '셈이', '수셈이', '수세미',
          '아마데우스최', '아마최','킹마최',
          '진희', '지니', '치니',
          '시리안','빡리안',
          '길버트','길벝','철도왕',
          '빅토리', '맑눈광', '토리',
          '설리반','리반이',
          '불곰',
          '샬롯', '성기사 샬롯',
          '버터우스', '버터',
          '크리즈','돌호동',
          '메카 맹기산','맹기산','기산이형','기사니','기싸니','기산',
          '아이스께끼', '께황','깨황','아이스께끼','께끼','아이쓰께끼','아이스께끼','깨끼',
          '썸네일','썸네일',"썸내일",'썸넬','썸낼'
         ]


isdol = ['이세돌','이세계 아이돌','챠니','챤이','비챤','릴파','르르땅','주르르','아잉네','아이네','세구','고세구','눈나구','지구즈','언니즈','막내즈','부산즈','개나리즈']

words = [
      (['우왁굳','왁굳','영택'],'Noun'), (['천양','대월향'],'Noun'),
      (isdol,'Noun'), (gomem,'Noun'),(akadam,'Noun')]

for word in words:
    name, poomsa = word
    twi.add_dictionary(name, poomsa)
    stopwords = ['의', '가', '은', '는','이', '과', '도', '를', '으로', '자', '에', '하고', '세요', '니다', '입니다',
                '하다', '을', '이다', '다', '것', '로', '에서', '그', '인', '서', '네요', '음', '임','랑',
                '게', '요', '에게', '엔', '이고', '거', '예요', '이에요', '어요', '어서', '여요', '하여']

text_token = []
for sentence in tqdm(comment_df['comment']):
    tmp = []
    tmp = twi.morphs(sentence, stem=True, norm=True)  #토큰화
    tmp = [word for word in tmp if not word in stopwords]  #불용어 제거
    text_token.append(tmp)


/usr/local/lib/python3.10/dist-packages/konlpy/tag/_okt.py:17: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')
100%|██████████| 1209/1209 [00:31<00:00, 38.94it/s] 


In [ ]:
comment_df['tmp'] = text_token

In [ ]:
comment_df['commentAt'] = pd.to_datetime(comment_df['commentAt'])
comment_df['year'] = comment_df['commentAt'].dt.year
comment_df['month'] = comment_df['commentAt'].dt.month

In [ ]:
comment_df

In [ ]:
# 단어를 통일시키는 함수
def unify_word(word, gomem_word):
    for unified_word, variations in gomem_word.items():
        if word in variations:
            return unified_word
    return word

def unify_tmp(tmp, gomem_word):
    return [unify_word(word, gomem_word) for word in tmp] # tmp 에 있는 word 들을



# 월별 고멤 언급량
def gomem_comment(df, col,month,year):
    if month == 'all':
        df = df[df['year'] == year]
    else:
        df = df[(df['year'] == year) & (df['month'] == month)]

    all_tmp = [word for sublist in df[col] for word in sublist] # word = 리스트 속 ['단어들']

    # 통일된 단어들만 추출 (gomem_word에 있는 단어들만 포함)
    # gomem_word와 aka_word에 속하는 단어들을 추출
    unified_tmp_gomem = unify_tmp(all_tmp, gomem_word)
    unified_tmp_gomem = [word for word in unified_tmp_gomem if word in gomem_word]

    unified_tmp_aka = unify_tmp(all_tmp, aka_word)
    unified_tmp_aka = [word for word in unified_tmp_aka if word in aka_word]

    # 각각의 단어 리스트를 Counter로 변환하여 가장 많이 나오는 단어들을 추출 top5
    most_gomem = Counter(unified_tmp_gomem).most_common(10)
    most_aka = Counter(unified_tmp_aka).most_common(10)

    return  most_gomem, most_aka

def wordCount(comment_df):
    all_tmp = [word for sublist in comment_df['tmp'] for word in sublist] # word = 리스트 속 ['단어들']

#   통일된 단어들만 추출
    unified_words = []
    for word in all_tmp:
        for unified_word, variations in word_rules.items():
            if word in variations:
                unified_words.append(unified_word)
                break

    unified_tmp = Counter(unified_words)
    most_common_words = unified_tmp.most_common(10)

    return most_common_words

def count_word(word_list, target_word):
    return word_list.count(target_word)

def gomem_video(df, gomem):
  # 단어통일
  df['tmp'] = df['tmp'].apply(lambda x: unify_tmp(x, word_rules))
  # '고멤' 단어가 언급된 빈도를 계산하여 데이터프레임에 추가
  df['cnt'] = df['tmp'].apply(lambda x: count_word(x, gomem))
  df = df.groupby(['video_id'])['cnt'].sum().reset_index() # comment_df 에 title 영상의 제목컬럼 필요
  gomem_hot_video = df[df['cnt'] == df['cnt']].nlargest(10,'cnt')[['video_id','cnt']]

  # 결과 출력
  return gomem_hot_video

In [ ]:
# 모든 멤버 한번에 월별로 보기

def gomem_comment_monthly_summary(df, col):
    # 월별 단어 집계를 저장할 리스트
    rows = []

    # 연도와 월별로 그룹화
    for (year, month), group in df.groupby(['year', 'month']):
        # group[col]에서 단어들을 추출
        all_tmp = [word for sublist in group[col] for word in sublist]

        # gomem_word와 aka_word에 속하는 단어들만 추출
        unified_tmp_gomem = unify_tmp(all_tmp, gomem_word)
        unified_tmp_gomem = [word for word in unified_tmp_gomem if word in gomem_word]

        # Counter로 단어별 빈도 계산
        word_counts = Counter(unified_tmp_gomem)

        # 결과를 한 행(row)으로 저장
        row = {'year': year, 'month': month}
        row.update(word_counts)  # 단어별 빈도를 추가
        rows.append(row)

    # 리스트를 DataFrame으로 변환
    result_df = pd.DataFrame(rows).fillna(0)  # NaN 값을 0으로 채움
    result_df = result_df.astype({'year': 'int', 'month': 'int'})  # 연도와 월을 정수형으로 변환

    # 열 순서를 정렬 (year, month 뒤에 단어들)
    fixed_columns = ['year', 'month']
    result_df = result_df[fixed_columns + [col for col in result_df.columns if col not in fixed_columns]]

    return result_df


In [ ]:
result_df = gomem_comment_monthly_summary(comment_df,'tmp')

In [ ]:
result_df

,year,month,캘리칼리,해루석,풍신,김치만두,왁파고,비즈니스킴,소피아,부정형 인간,...,히키킹,비밀소녀,곽춘식,단답벌레,권민,바이터,뢴트게늄,이덕수,융터르,티파니
0,2023,12,376,627,132,158,166,458,284,51,...,899,201,292,220,321,53,356,19,240,1.0
1,2024,1,289,897,139,96,257,708,519,37,...,754,286,137,154,669,83,769,59,38,0.0
2,2024,2,162,675,53,60,353,187,159,24,...,369,74,321,189,190,110,409,190,134,0.0
3,2024,3,49,198,19,164,121,314,440,385,...,217,211,188,39,246,502,326,194,57,0.0
4,2024,4,121,592,51,64,433,500,391,86,...,1212,182,338,56,565,744,544,125,188,1369.0
5,2024,5,68,455,171,84,134,215,80,96,...,251,734,380,318,113,398,125,53,637,133.0
6,2024,6,86,257,40,77,474,100,468,72,...,194,123,35,299,93,270,387,65,82,755.0
7,2024,7,128,651,46,294,142,226,307,31,...,212,136,464,203,47,442,919,38,179,583.0
8,2024,8,68,182,304,155,67,536,115,109,...,109,110,52,60,200,114,306,101,64,219.0
9,2024,9,227,153,65,72,94,110,304,416,...,87,54,113,233,90,104,263,46,85,128.0


In [ ]:
gomem = result_df.copy()
# 합계 행 추가 (열별 합계)
gomem.loc['합계'] = gomem.iloc[:, 2:].sum()  # 2번째 열부터 끝까지 합계 계산

# 합계 기준 정렬
sorted_totals = gomem.loc['합계'].sort_values(ascending=False)
sorted_totals
# 출력 결과 확인
# sorted_totals = pd.DataFrame(sorted_totals, columns=['합계']).reset_index()
# sorted_totals.columns = ['멤버', '합계']
# sorted_totals

,합계
해루석,4966.0
뢴트게늄,4694.0
도파민,4648.0
히키킹,4632.0
비즈니스킴,3629.0
소피아,3369.0
티파니,3305.0
바이터,2974.0
권민,2661.0
곽춘식,2625.0


In [ ]:
# 2024년 전체
most_gomem, most_ak = gomem_comment(comment_df,'tmp','all',2024)

In [ ]:
most_gomem

[('해루석', 4339),
 ('뢴트게늄', 4338),
 ('도파민', 4248),
 ('히키킹', 3733),
 ('티파니', 3304),
 ('비즈니스킴', 3171),
 ('소피아', 3085),
 ('바이터', 2921),
 ('권민', 2340),
 ('곽춘식', 2333)]

In [ ]:
# 멤버의 hot video
hot_video = gomem_video(comment_df,'썸네일')

In [ ]:
hot_video

['TKNWBrnXCT0',
 'Hb-Zp9ExPYY',
 'V9sxbdwK4Os',
 'dG_xwjn0yeI',
 'R8qftMjv6x0',
 '67O2YGMn7L4',
 't4AkamgLfx8',
 'U7u88wU590c',
 'tzFgjXXMUP4',
 'a9wknKtXQa4']

In [ ]:
video_ids = hot_video['video_id'].tolist()
thumb_comment = comment_df[comment_df['video_id'].isin(video_ids)][['author','author_channel_id','comment','like','commentAt','video_id','title','tmp']]# '썸네일'이 tmp 컬럼에 포함된 행만 필터링
filtered_df = thumb_comment[thumb_comment['tmp'].apply(lambda tokens: '썸네일' in tokens)]

# video_id 별로 상위 10개 행만 선택
top10_per_video = (
    filtered_df.groupby('video_id', group_keys=False)  # video_id로 그룹화
               .apply(lambda group: group.sample(10))   # 각 그룹에서 상위 10개 행 선택
)

top10_per_video


In [ ]:
comment_df[comment_df['video_id'].isin(video_ids)].groupby(['video_id','title']).count().reset_index()[['video_id','title']]

,video_id,title
0,-CDAA7r2aIs,SKYSIX (육광천) 'Everything' Official MV
1,FSxe7lXhz6g,러브픽션 - Cover By 뢴트게늄
2,N0yaNNbSAf8,어항 [을 깨다:부시다] - 뢴트게늄 작곡캠프 오리지날 송
3,U7u88wU590c,대이변이 일어났습니다 (인기투표 1위 발표)
4,UJNfIQrw9jM,なとり - Overdose(오버도즈)ㅣ뢴트게늄 Cover
5,ULkBnaT7ivk,뢴트게늄&해루석&히키킹 - 뢴해히 CYPHER ｜왁타버스 뮤직 어워즈 2023


In [ ]:
# 영상에서 가장 많이 언급된 고멤
# video_ids
video_by_gomem = wordCount(comment_df)
pd.DataFrame(video_by_gomem).head(5)

,0,1
0,도파민,25
1,비즈니스킴,5
2,독고혜지,4
3,융터르,1
4,진희,1


## href 댓글 언급 구간 가져오기

In [ ]:
# top20 비디오 아이디
top20= pd.read_csv(io.StringIO('''
zJV36hd8xtY
Hb-Zp9ExPYY
mFVOWiPHacM
MCswDuAYSpM
a9p1ie7hWzU
U7u88wU590c
a9wknKtXQa4
gFiiA2lvSBQ
2zqD0izjIFw
GG3-K6g_Sco
hu9NQsVPEBE
tZ9Lb1dWLTc
Ulez7NEsZJg
btLCrhQDlDU
jeaX-SNRbbY
OBvdY6SLU9k
nlGTh9LzYpU
TVy3fDXRuUQ
iQdn6vmhJVc
VV8GLpxxW_g
'''), header=None)
video_ids= top20[0].tolist()
len(video_ids)

20

In [ ]:
# 유튜브 댓글 데이터
comment_df = pd.read_csv('/content/drive/MyDrive/WAKTUBE/24_wakgood_comment_final.csv')

In [ ]:
# top20에 해당하는 비디오 댓글만
comment_df = comment_df[comment_df['video_id'].isin(video_ids)]

In [ ]:
# youtube timeline 의 경우, </a> 태그로 감싸져있음.

def extract_time_info(comment):
  time_pattern = r"(\d+:\d+)</a>"
  time_info = re.findall(time_pattern, comment)
  return time_info

comment_df['time_info'] = comment_df['comment'].apply(extract_time_info)
comment_df['time_info'] = comment_df['comment'].apply(lambda x: [x for x in extract_time_info(x)]) # 리스트 타입으로

In [ ]:
# time line 언급이 있는 댓글만 가져오기
timeline_df = comment_df[comment_df['time_info'].apply(lambda x: len(x) > 0)]

In [ ]:
from collections import Counter

# 시간 정보를 초 단위로
def time_to_seconds(time_str):
    minutes, seconds = map(int, time_str.split(':'))
    return minutes * 60 + seconds

# video_id 별 집계
def aggregate_by_video_id(timeline_df):
    results = []

    # video_id 별로 그룹
    grouped = timeline_df.groupby('video_id')
    for video_id, group in grouped:
        # 모든 시간 정보를 리스트로 풀어내기
        all_times = [time for times in group['time_info'] for time in times]

        # 초 단위로 변환
        time_in_seconds = [time_to_seconds(time) for time in all_times]

        # 10초 단위로 그룹화
        time_grouped = [10 * (time // 10) for time in time_in_seconds]

        # 빈도 계산
        time_count = Counter(time_grouped)

        # 결과를 DataFrame으로 정리
        video_time_df = pd.DataFrame(time_count.items(), columns=['seconds', 'count'])
        video_time_df = video_time_df.sort_values(by='seconds')
        video_time_df['video_id'] = video_id  # video_id 추가

        results.append(video_time_df)

    # 모든 결과를 하나의 DataFrame으로 병합
    final_df = pd.concat(results, ignore_index=True)
    return final_df

# timeline_df에 대해 video_id 별 집계 수행
time_df = aggregate_by_video_id(timeline_df)


In [ ]:
time_df

,seconds,count,video_id
0,10,4,WVLV_BtT0xQ
1,80,1,WVLV_BtT0xQ
2,170,1,WVLV_BtT0xQ
3,180,1,WVLV_BtT0xQ
4,240,1,WVLV_BtT0xQ
5,250,4,WVLV_BtT0xQ
6,270,2,WVLV_BtT0xQ
7,280,1,WVLV_BtT0xQ
8,300,3,WVLV_BtT0xQ
9,310,2,WVLV_BtT0xQ


In [ ]:
sample = timeline_df[timeline_df['video_id'] == 'GG3-K6g_Sco']

# 시간 정보 통합(분 기준)
all_times = [time for times in sample['time_info'] for time in times]
minute_only = [int(time.split(':')[0]) for time in all_times]
# 구간별 빈도 계산
interval_counts = Counter(minute_only)
highlight, highlight_cnt = interval_counts.most_common(1)[0]
highlight_seconds = highlight * 60

In [ ]:
# 10초 단위로 집계
def time_to_seconds(time_str):
    minutes, seconds = map(int, time_str.split(':'))
    return minutes * 60 + seconds

time_in_seconds = [time_to_seconds(time) for time in all_times]

# 10초 단위로 그룹화
time_grouped = [10 * (time // 10) for time in time_in_seconds]

# 빈도 계산
time_count = Counter(time_grouped)

time_df = pd.DataFrame(time_count.items(), columns=['seconds', 'count'])
time_df = time_df.sort_values(by='seconds')
time_df

In [ ]:
interval_counts_list = [{'minute': minute, 'count': count} for minute, count in interval_counts.items()]
interval_df = pd.DataFrame(interval_counts_list)
interval_df.sort_values(by='minute', ascending=True)

,minute,count
19,0,11
9,1,26
17,2,26
15,3,51
0,4,19
8,5,32
11,6,31
14,7,18
18,8,6
22,9,1


# 통계 실험소 🧪

## 고멤들의 썸네일에 따라 조회수 차이가 있을까?

In [ ]:
df = pd.read_csv('/content/2024 썸네일 데이터 - 시트16.csv')

In [ ]:
# 집계 결과를 저장할 딕셔너리 생성
view_count_summary = []

# 멤버 컬럼 리스트 추출 (닌닌, 빅토리, 세용, ...)
member_columns = [col for col in df.columns if col not in ['view_count', 'view_count_N']]

# 각 멤버 컬럼별로 view_count_N 합계, 평균, 개수 계산
for col in member_columns:
    member_data = df.loc[df[col] == 1, 'view_count_N']
    view_count_summary.append({
        'Member': col,
        'Total_view_sum': member_data.sum(),
        'Total_view_mean': member_data.mean(),
        'Total_view_count': member_data.count()
    })

# 결과를 데이터프레임으로 변환
summary_df = pd.DataFrame(view_count_summary)


In [ ]:
summary_df.sort_values(by='Total_view_sum', ascending=False)

,Member,Total_view_sum,Total_view_mean,Total_view_count
21,바이터,16310827,776706.080952,21
9,해루석,9984920,192017.708654,52
1,빅토리,9365443,187308.866667,50
0,닌닌,7382470,230702.203126,32
12,뢴트게늄,6733329,164227.556911,41
5,시리안,6342360,218702.071839,29
7,독고혜지,5815731,232629.254000,25
3,쵸로키,5708146,285407.308335,20
2,세용,5226708,153726.732845,34
13,히키킹,5063317,168777.252778,30


In [ ]:
# 주어진 데이터를 리스트로 저장
data = [
    ['여', '아카', '빅토리'],
    ['여', '아카', '닌닌'],
    ['여', '아카', '세용'],
    ['남', '아카', '젠투'],
    ['여', '아카', '시리안'],
    ['남', '아카', '아마최'],
    ['남', '아카', '진희'],
    ['여', '아카', '쵸로키'],
    ['남', '아카', '설리반'],
    ['여', '아카', '샬롯'],
    ['남', '아카', '맹기산'],
    ['남', '아카', '버터우스'],
    ['남', '아카', '불곰'],
    ['남', '아카', '수셈이'],
    ['남', '아카', '크리즈'],
    ['남', '아카', '길버트'],
    ['남', '아카', '께끼'],
    ['남', '고멤', '해루석'],
    ['남', '고멤', '뢴트게늄'],
    ['남', '고멤', '왁파고'],
    ['남', '고멤', '히키킹'],
    ['여', '고멤', '독고혜지'],
    ['남', '고멤', '도파민'],
    ['남', '고멤', '비즈니스킴'],
    ['남', '고멤', '바이터'],
    ['남', '고멤', '소피아'],
    ['남', '고멤', '권민'],
    ['남', '고멤', '융터르'],
    ['남', '고멤', '캘리칼리'],
    ['남', '고멤', '곽춘식'],
    ['남', '고멤', '이덕수'],
    ['여', '고멤', '비밀소녀'],
    ['여', '고멤', '티파니'],
    ['남', '고멤', '풍신'],
    ['남', '고멤', '부정형'],
    ['남', '고멤', '김치만두'],
    ['남', '고멤', '단답벌레'],
    ['남', '고멤', '호드']
]

# 데이터프레임 생성
member_info_df = pd.DataFrame(data, columns=['성별', '소속', 'Member'])
# summary_df와 member_info_df를 멤버 컬럼 기준으로 merge
merged_df = summary_df.merge(member_info_df, how='left', left_on='Member', right_on='Member')


In [ ]:
# 유치원, 이세돌은 제외
merged_df

,Member,Total_view_count_N,성별,소속
0,바이터,776706.080952,남,고멤
1,수셈이,493159.937500,남,아카
2,티파니,403609.287909,여,고멤
3,단답벌레,402391.083325,남,고멤
4,비밀소녀,355408.571795,여,고멤
5,쵸로키,285407.308335,여,아카
6,권민,242008.217592,남,고멤
7,맹기산,237001.763636,남,아카
8,독고혜지,232629.254000,여,고멤
9,풍신,231613.988121,남,고멤


In [ ]:
# 성별별로 총 조회수 합계, 평균을 집계
view_by_gender = merged_df.groupby('성별').agg(
    avg_view=pd.NamedAgg(column='Total_view_count_N', aggfunc='mean')   # 조회수 평균
).reset_index()


view_by_gender

,성별,avg_view
0,남,189848.981645
1,여,244104.550040


In [ ]:
from scipy import stats

# 성별별 조회수 데이터 추출
female_views = merged_df[merged_df['성별'] == '여']['Total_view_count_N']
male_views = merged_df[merged_df['성별'] == '남']['Total_view_count_N']

# 독립 표본 t-검정
t_stat, p_value = stats.ttest_ind(female_views, male_views, equal_var=False)

# 결과 출력
print(f"t-statistic: {t_stat}")
print(f"p-value: {p_value}")

# 유의수준 0.05에서 검정 결과 해석
if p_value < 0.05:
    print("성별에 따른 조회수 차이는 통계적으로 유의미합니다.")
else:
    print("성별에 따른 조회수 차이는 통계적으로 유의미하지 않습니다.")


t-statistic: 1.3426579324057788
p-value: 0.19305065650451536
성별에 따른 조회수 차이는 통계적으로 유의미하지 않습니다.
